## Feature Selection and Lasso Regression on House Sales Data

### Fire up Graphlab Create

In [1]:
import graphlab

### Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to agrawal.pr@husky.neu.edu and will expire on March 12, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\agraw\AppData\Local\Temp\graphlab_server_1504744451.log.0


### Explore house sales data

In [7]:
sales[0:1]

Columns:
	id	str
	date	datetime
	price	float
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	int
	floors	str
	waterfront	int
	view	int
	condition	int
	grade	int
	sqft_above	int
	sqft_basement	int
	yr_built	int
	yr_renovated	int
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float

Rows: 1

Data:
+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |
+-------------+----------+--------+------------+------+-----------+-------+------------+
+---------------+----------+--------------+---------+-------------+
| sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+---------------+----------+--------------+---------+-------------+
|       0       |   1955   |      0       |  98178  | 47.51123398 |
+---------------+----------+--------------+---------+-------------+
+---------------+---------------+-----+
|      long     | sqft_living15 | ... |
+---------------+---------------+-----+
| -122.25677536 |     1340.0    | ... |
+---------------+---------------+-----+
[1 rows x 21 columns]

### Create new features

In [8]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

### Learn regression weights with L1 penalty

In [9]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [10]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.541104     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.574187     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [11]:
model_all

Class                          : LinearRegression

Schema
------
Number of coefficients         : 18
Number of examples             : 21613
Number of feature columns      : 17
Number of unpacked features    : 17

Hyperparameters
---------------
L1 penalty                     : 10000000000.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : fista
Solver iterations              : 10
Solver status                  : TERMINATED: Iteration limit reached.
Training time (sec)            : 1.6855

Settings
--------
Residual sum of squares        : 2.84262903437e+15
Training RMSE                  : 362662.4299

Highest Positive Coefficients
-----------------------------
(intercept)                    : 274873.056
bathrooms                      : 8468.5311
grade                          : 842.068
sqft_living_sqrt               : 350.0606
sqft_living                    : 24.4207

Lowest Negative Coefficients
----------------------------
No Ne

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection.

### Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split sales data into 2 sets: training and test
* Further split training data into two sets: train, validation

In [12]:
(training_and_validation, testing) = sales.random_split(.9,seed=1)
(training, validation) = training_and_validation.random_split(0.5, seed=1)

* Fit a regression model with a given `l1_penalty` on TRAIN data. 
* Compute the RSS on VALIDATION data for that `l1_penalty`
* Report the `l1_penalty` that produces the lowest RSS on validation data.

In [24]:
import numpy as np
lowest_rss = None
best_l1_penalty = 0.0
for l1_penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l1_penalty=l1_penalty, l2_penalty=0, verbose=False)
    predicted_output = model.predict(validation)
    residuals = predicted_output - validation['price']
    rss = (residuals*residuals).sum()
    if lowest_rss == None:
        lowest_rss = rss
        best_l1_penalty = l1_penalty    
    elif rss < lowest_rss:
        lowest_rss = rss
        best_l1_penalty = l1_penalty

In [25]:
best_l1_penalty

10.0

### Limit the number of nonzero weights

1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region found to find a good value for `l1_penalty` that achieves the desired sparsity.

In [44]:
max_nonzeros = 7

### Exploring the larger range of values to find a narrow range with the desired sparsity

In [27]:
l1_penalty_values = np.logspace(8, 10, num=20)

In [34]:
non_zeros = []
l1_penalties = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l1_penalty=l1_penalty, l2_penalty=0, verbose=False)
    non_zeros.append(model['coefficients']['value'].nnz())
    l1_penalties.append(l1_penalty)

In [36]:
print non_zeros

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

* The largest `l1_penalty` that has more non-zeros than `max_nonzeros`
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros'

In [40]:
print l1_penalties

[100000000.0, 127427498.57031322, 162377673.91887242, 206913808.11147901, 263665089.87303555, 335981828.62837881, 428133239.8719396, 545559478.11685145, 695192796.17755914, 885866790.41008317, 1128837891.6846883, 1438449888.2876658, 1832980710.8324375, 2335721469.0901213, 2976351441.6313128, 3792690190.7322536, 4832930238.5717525, 6158482110.6602545, 7847599703.5146227, 10000000000.0]


In [41]:
l1_penalty_min = l1_penalties[15]
l1_penalty_max = l1_penalties[14]

### Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [43]:
l1_penalty_values = np.linspace(l1_penalty_min, l1_penalty_max, 20)

In [46]:
lowest_rss = None
best_model = None
best_l1_penalty = 0.0
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l1_penalty=l1_penalty, l2_penalty=0, verbose=False)
    predicted_output = model.predict(validation)
    residuals = predicted_output - validation['price']
    rss = (residuals*residuals).sum()
    if lowest_rss == None:
        lowest_rss = rss
        best_l1_penalty = l1_penalty
        best_model = model
    elif rss < lowest_rss:
        lowest_rss = rss
        best_l1_penalty = l1_penalty
        best_model = model

### features in the model that have non-zero coefficients

In [47]:
best_model['coefficients']

name,index,value,stderr
(intercept),None,196100.937806,None
bedrooms,None,2181.57432107,None
bedrooms_square,None,0.0,None
bathrooms,None,17962.6966612,None
sqft_living,None,34.1424656512,None
sqft_living_sqrt,None,789.319789078,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,3665.9308176,None
floors_square,None,0.0,None
